In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import os
from glob import glob
import re

# Carpeta que contiene los archivos Excel
carpeta = "."
archivos = glob(os.path.join(carpeta, "*.xlsx"))

# Función de clasificación
def clasificar(respuesta):
    val = str(respuesta).strip()
    if val in ["1", "2"]:
        return "Correcta"
    elif val == "0":
        return "Incorrecta"
    elif val == "-":
        return "Sin contestar"
    else:
        return "Nula"

# Función para extraer número de la pregunta para ordenamiento
def extraer_numero(pregunta):
    match = re.search(r'\d+', str(pregunta))
    return int(match.group()) if match else float('inf')

# Carpeta de salida para gráficos
output_folder = "graficos"
os.makedirs(output_folder, exist_ok=True)

# Procesar cada archivo
for archivo in archivos:
    try:
        df = pd.read_excel(archivo)
        nombre_archivo = os.path.basename(archivo)
        nombre_base = os.path.splitext(nombre_archivo)[0]

        # Verificación básica
        if "Apellido(s)" not in df.columns or "Nombre" not in df.columns:
            print(f"[{nombre_archivo}] Archivo inválido. Saltando.")
            continue

        df["Nombre completo"] = df["Apellido(s)"] + " " + df["Nombre"]

        # Detectar columna de calificación
        col_calif = next((col for col in df.columns if col.startswith("Calificación")), None)
        preguntas_cols = [col for col in df.columns if col.startswith("P.") or col.startswith("P ")]

        id_vars = ["Nombre completo"]
        if col_calif:
            id_vars.append(col_calif)

        # Despivot
        df_unpivot = df.melt(
            id_vars=id_vars,
            value_vars=preguntas_cols,
            var_name="Pregunta",
            value_name="Respuesta"
        )

        df_unpivot["Respuesta Tipo"] = df_unpivot["Respuesta"].apply(clasificar)

        # Agrupar para gráfico
        conteo = df_unpivot.groupby(["Pregunta", "Respuesta Tipo"]).size().unstack(fill_value=0)

        # Reordenar preguntas por número
        conteo = conteo.loc[sorted(conteo.index, key=extraer_numero)]

        # Graficar
        ax = conteo.plot(
            kind="bar",
            stacked=True,
            figsize=(12, 5),
            title=f"Respuestas por pregunta - {nombre_base}",
            color={"Correcta": "#09f", "Incorrecta": "red", "Sin contestar": "gray", "Nula": "orange"}
        )
        plt.xlabel("Pregunta")
        plt.ylabel("Cantidad de respuestas")
        plt.legend(title="Tipo de respuesta")
        plt.tight_layout()

        # Guardar el gráfico
        output_path = os.path.join(output_folder, f"{nombre_base}.png")
        plt.savefig(output_path)
        plt.close()
        print(f"[{nombre_archivo}] Gráfico generado en: {output_path}")

    except Exception as e:
        print(f"[{archivo}] Error al procesar: {e}")


[1_ciclo_matematica.xlsx] Gráfico generado en: graficos/1_ciclo_matematica.png
[1 CICLO DIAGNOSTICO GRAFICO LENGUAJE.xlsx] Gráfico generado en: graficos/1 CICLO DIAGNOSTICO GRAFICO LENGUAJE.png
[II CICLO SOCIALES.xlsx] Gráfico generado en: graficos/II CICLO SOCIALES.png
[2 ciclo lenguaje.xlsx] Gráfico generado en: graficos/2 ciclo lenguaje.png
[II CICLO MATEMATICA.xlsx] Gráfico generado en: graficos/II CICLO MATEMATICA.png
[1 CICLO CIENCIAS NATURALES.xlsx] Gráfico generado en: graficos/1 CICLO CIENCIAS NATURALES.png
